In [41]:
# svm classifer to classify the AD and NC

# 1. load the data and match the outpiut labels with the feature vectors
# from data 0 - 39 
# that csv is in starting from index 40 
# Downloads/data_labels.csv
from csv import reader
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


# read the output labels 
label_data={}
with open("data_labels.csv", "r") as label_file:
    label_csv=reader(label_file,delimiter=",")
    #skip header row
    next(label_csv)

    count=0
    for line in label_csv:
        if count >= 40:
            break
        #brain label
        brain_label = line[1].strip()
        brain_idx=re.findall(r'\d+',brain_label)
        brain_idx_str=''.join(brain_idx)
        # print(brain_idx_str)
        output_label = line[2].strip().lower()
        #output label
        label_data[brain_idx_str] = 1  if output_label == "ad" else 0

        count +=1 #increment counter

#print the output label disctionary
# print(label_data)

# open grey matter volume csv
with open("AAL_statistics_volume.csv") as f:
    csv_data=reader(f,delimiter=',')
    raw_data=np.array(list(csv_data),dtype=str)

#preprocess data
data_x = []
data_y = []
data_to_predict=[] #feature vectors with unknown labels
ids_to_predict=[]

for row in raw_data:
    if row.size==0: #check if row is empty
        continue
    # skip first column and extarct first val from each column
    # ignore the data label 
    feature_vals= [float(cell.split()[0]) for cell in row[1:] if cell.strip()]
    # print(len(feature_vals))
    # print(len(feature_vals))

    #use the brain label to fetch corresponding label
    brain_label = row[0].strip()
    brain_idx=re.findall(r'\d+',brain_label)
    brain_idx_str=''.join(brain_idx)
    
    if brain_idx_str in label_data:
        data_x.append(feature_vals)#append to the feature list
        data_y.append(label_data[brain_idx_str])
    else:
        data_to_predict.append(feature_vals) #add data without labels
        ids_to_predict.append(brain_idx_str)

data_x = np.array(data_x)
data_y =np.array(data_y)
data_to_predict = np.array(data_to_predict)

# print(len(data_y))
# print(len(data_x))
# print(len(data_to_predict))


#continue with feature normalisation 
scaler=StandardScaler()
#transform all feature data
all_data = np.vstack((data_x, data_to_predict))
all_data_normalized = scaler.fit_transform(all_data)

#split the normalised data into known and unknown labels
X_train=all_data_normalized[:len(data_x)]
X_predict=all_data_normalized[len(data_x):]


#train SVM classifier
svm_classifier=SVC(kernel='rbf', C=1.0, gamma="scale")
svm_classifier.fit(X_train,data_y)

#predict the labels for unknown data points
predicted_labels=svm_classifier.predict(X_predict)

#print predicted output
for identifier, label in zip(ids_to_predict, predicted_labels):
    print(f"ID: {identifier}, Predicted Label: {'AD' if label == 1 else 'NC'}")



ID: 40, Predicted Label: NC
ID: 41, Predicted Label: NC
ID: 42, Predicted Label: AD
ID: 43, Predicted Label: NC
ID: 44, Predicted Label: AD
ID: 45, Predicted Label: AD
ID: 46, Predicted Label: AD
ID: 47, Predicted Label: AD
ID: 48, Predicted Label: AD
ID: 49, Predicted Label: NC
